<h1> Hyperledger Sawtooth </h1>

* Exemplos de implementações:
<ol>
<li><a href="https://github.com/yellowssi/SeaStorage">Implementação decentralizada para file storage</a></li>
<li><a href="https://github.com/Kerala-Blockchain-Academy/hygieia">Implementação de saúde (doação de orgãos)</a></li>
<li><a href="https://github.com/hyperledger-archives/sawtooth-marketplace"> Implementação marketplace</li>

<li><a href="https://github.com/hyperledger-archives/sawtooth-supply-chain">Supply chain</li>

<li><a href="https://github.com/hyperledger-archives/sawtooth-private-utxo">Sei la o que é isso</li>

<li></li>
</ol>


<a href="https://events19.linuxfoundation.org/wp-content/uploads/2017/12/Blockchain-Technology-Partners-OSS-EU-2018-Session_Duncan-Johnston-Watt.pdf
">Documento pode ajudar a entender melhor como o VALIDADOR  funciona</a>

Aparentemente é possível utilizar um validador para diversas aplicações (TransactionProcessors)-> Definindo no validador uma transaction family para cada aplicação.

--> Discussão sobre o design de dados escolhido no sawtooth: https://www.lfdecentralizedtrust.org/blog/2018/01/16/data-model-considerations-in-hyperledger-sawtooth


* Aparentemente o exemplo da simple-wallet parece ser útil e simples o suficiente para explorar: https://github.com/askmish/sawtooth-simplewallet

<h1> Hyperledger fabric </h1>

* Parece simples, mais utilizado e tem mais tutorials

* Dois algoritmos de consenso: pBFT e RAFT

* Pelo que entendi, o consenso ocorre no orderer. ?? Porém o consenso é apenas sobre a ordem em que as transações são armazenadas em um bloco ?? No entanto, garantir a ordem já é suficiente para nosso modelo ! pois queremos que todos os nós envolvidos informem seu QoS de forma honesta.

* problema: nós precisam se conhecer antes de formar a blockchain...

* Limitation of Hyperledger Fabric

    1 Hyperledger Fabric is a robust and flexible platform for developing blockchain applications, but like any technology, it has certain limitations:

    2 Scalability: Hyperledger Fabric is designed for permissioned networks, where the participants are known and trusted, which can limit its scalability for large-scale public networks.
    
    3 Performance: The performance of Hyperledger Fabric can be impacted by factors such as network size, network configuration, and the complexity of chaincode, which can limit its ability to handle high volumes of transactions.

    4 Complexity: Setting up and configuring a Hyperledger Fabric network can be complex, requiring a deep understanding of the technology and its components.
    
    5 Compatibility: Hyperledger Fabric is designed to be used with specific programming languages, such as Go and JavaScript, which can limit its compatibility with other technologies and programming languages.
    
    6 Cost: Running a Hyperledger Fabric network requires infrastructure and resources, which can add costs to the deployment and operation of blockchain applications.

    7 Interoperability: Hyperledger Fabric is designed to be used within a single network, and its interoperability with other blockchain platforms is limited.

<h1> O que eu quero </h1>

* Uma blockchain onde cada ledger tenha um conjunto de nós validadores, que podem mudar com o tempo.

* Ou, uma blockchain onde cada blockchain tenha um conjunto de nós validadores que podem mudar com o tempo.

* O que percebi, redes permissionadas precisa-se conhecer os nós de antemão e as não permissionadas aparentemente são mais livres dessa limitação.

* Get dockerfile of an image: sudo docker history --no-trunc 7d249ca8774e >> sawtooth_validator_dockerfile.dockerfile


<h2> NEXT </h2>

* Combinar os diversos containers em um apenas (validador, settings, rest, consensus) --> diminuir exposição de portas e permitir mais blcockchains ++ diminuir recursos por blockchain.

<H3> Comandos docker CLI <H3>

- Ver todos os containers (parados tbm):

```
sudo docker ps -a
```

- Criar imagem a partir de dockerfile:

```
sudo docker build --debug --tag 'helloworldv1' .
```

- listar containers rodando:

```
sudo docker container ls
```

- iteragir com container:

```
sudo docker exec -it b6575d8802fd sh
```

<h2>Como e onde são armazenados os blocos</h2>

* para referencia futura:: (https://stackoverflow.com/questions/59029691/hyperledger-fabric-where-the-blockchain-state-are-saved)

You will need to use persistent volumes to ensure that the data is not stored on the container filesystem else it will be destroyed when the container(s) are destroyed.

For peers, the two key attributes in core.yaml are:

peer.fileSystemPath - this defaults to /var/hyperledger/production and is where the ledger, installed chaincodes, etc are kept. The corresponding environment variable is CORE_PEER_FILESYSTEMPATH.

peer.mspConfigPath - where the local MSP info is stored. The corresponding environment variable is CORE_PEER_MSPCONFIGPATH.

For orderers, the two key attributes in orderer.yaml are:

FileLedger.Location - this defaults to /var/hyperledger/production/orderer and is where the channel ledgers are stored. The corresponding environment variable is ORDERER_FILELEDGER_LOCATION.

General.LocalMSPDir - where the local MSP info is stored. The corresponding environment variable is ORDERER_GENERAL_LOCALMSPDIR.

<h2>Explicação de como os blocos são armazenados</h2

(https://learnthings.online/course/2020/03/06/hyperledger-sawtooth-for-application-developers)

State Encoding

Applications serialize payloads to make the data “opaque” to the core Sawtooth system. The validator sees the data as simply a string of bytes. Only the transaction processor that handles the transaction will deserialize the payload.

When interacting with the blockchain, a transaction processor sets and retrieves state data by making calls against a version of state that the validator provides.

get_state(address) returns the byte array found at the address
set_state(address, data) sets the byte array stored at that address.
Likewise, when a client sends a transaction to the validator, it must serialize the payload data.

For this reason, the encoding scheme must be deterministic; serialization and deserialization must always produce the exact same results. Even the slightest difference in the state entities across platforms or executions (such as the keys being in a different order or rounding inconsistencies for floating-point numbers) can break the global state agreement. Avoid data structures that don’t enforce an ordered serialization (such as sets, maps, dicts, or JSON objects). Also take care to avoid data types that may be interpreted differently across platforms.

Simple Supply uses protocol buffers (protobufs) to encode all objects before storing them in state (both payloads and state data. Using protobufs here is convenient because Simple Supply uses protobufs to serialize batches and transactions. Although protobufs don’t fully guarantee determinism, they serialize and deserialize identically for the purposes of Simple Supply. The Simple Supply protobuf messages are in education-sawtooth-simple-supply/protos.

<h2>Histórico de mudanças de estado</h2>

https://stackoverflow.com/questions/52328352/changes-made-to-an-address-in-sawtooth

-> precisa reconstruir o historico percorrendo todos os blocos (ou se vc souber em qual bloco todas as informações foram armazenadas).

<h3>Rest api source code</h3>

* Caso queira modificar algum comportamento ou adicionar funcionalidade:

<a href="https://github.com/hyperledger-archives/sawtooth-core/blob/main/rest_api/sawtooth_rest_api/rest_api.py">github</a>


Routes da API:

    app.router.add_post('/batches', handler.submit_batches)
    app.router.add_get('/batch_statuses', handler.list_statuses)
    app.router.add_post('/batch_statuses', handler.list_statuses)

    app.router.add_get('/state', handler.list_state)
    app.router.add_get('/state/{address}', handler.fetch_state)

    app.router.add_get('/blocks', handler.list_blocks)
    app.router.add_get('/blocks/{block_id}', handler.fetch_block)

    app.router.add_get('/batches', handler.list_batches)
    app.router.add_get('/batches/{batch_id}', handler.fetch_batch)

    app.router.add_get('/transactions', handler.list_transactions)
    app.router.add_get(
        '/transactions/{transaction_id}',
        handler.fetch_transaction)

    app.router.add_get('/receipts', handler.list_receipts)
    app.router.add_post('/receipts', handler.list_receipts)

    app.router.add_get('/peers', handler.fetch_peers)
    app.router.add_get('/status', handler.fetch_status)

    subscriber_handler = StateDeltaSubscriberHandler(connection)
    app.router.add_get('/subscriptions', subscriber_handler.subscriptions)

<h3>TODO</h3>

* Compilar o TransactionProcessor para colocar no container, pois não tem Python instalado lá (economia de recursos)